In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

# Loading the data

In [4]:
df = pd.read_pickle("data/data_for_ml.pkl")
df.head()

,Temperatura a 2m °C,velocidade do vento a 10m,umidade relativa a 2 m,pressão hpa,WPD,ano,dia,hora
0,26.43,4.911,69.77,1009.2,117.230155,2004,1,100
1,26.04,4.995,72.70,1009.5,123.526100,2004,1,200
2,25.57,3.355,77.10,1009.7,37.486827,2004,1,300
3,25.23,2.798,78.30,1009.8,21.771341,2004,1,400
4,25.10,2.744,78.10,1010.4,20.557634,2004,1,500


In [48]:
df_train = df[(df["ano"]!=2007) & (df["ano"]!=2006) ]

df_test = df[df["ano"]==2006]

In [49]:
X_train = df_train.iloc[:,:4].values
y_train = df_train.iloc[:,4].values
X_test = df_test.iloc[:,:4].values
y_test= df_test.iloc[:,4].values

In [50]:
y_test.mean()

100.5732584091019

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
variable_objetivo = "WPD"
variables_independientes = df.drop(variable_objetivo, axis=1).columns

In [10]:
df_x = df[variables_independientes]
df_y = df[variable_objetivo]

# Standardizing the data

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
escalador = StandardScaler()
variables_imputadas =  escalador.fit_transform(df_x)


In [7]:
df_x_imputado = pd.DataFrame(variables_imputadas, columns = variables_independientes)
df_x_imputado.head()

,Temperatura a 2m °C,velocidade do vento a 10m,umidade relativa a 2 m,pressão hpa
0,-0.401849,0.496924,0.056244,-0.793673
1,-0.544353,0.536412,0.272848,-0.663258
2,-0.716089,-0.234538,0.598122,-0.576315
3,-0.840323,-0.496379,0.686833,-0.532844
4,-0.887825,-0.521764,0.672048,-0.272014


**I define rmse for cross_val_score:**

In [22]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmse_cv(estimador, X, y):
    preds = estimador.predict(X)
    return rmse(y, preds)


**I create the following functions to evaluate the model and put the results in the same table**

In [23]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring= rmse_cv, n_jobs=-1, cv=10, return_train_score=True)# roc_auc por que esta muy desvalanceada y no le importa tanto
    return resultados_estimador

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].max()
    return resultados_df

**I create an emptu dictionary to introduce the results**

In [24]:
resultados = {}

**importing the algorithms**

In [25]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.svm import  SVR

In [26]:
resultados["elastic"] = evaluar_modelo(ElasticNet(random_state=42), X_train, y_train)
resultados["Lasso"] = evaluar_modelo(Lasso(random_state=42), X_train, y_train)
resultados["Ridge"] = evaluar_modelo(Ridge(random_state=42),  X_train, y_train)
resultados["Linear"] = evaluar_modelo(LinearRegression(),  X_train, y_train)
resultados["SVR"] = evaluar_modelo(SVR(), X_train, y_train)
resultados["KNN"] = evaluar_modelo(KNeighborsRegressor(), X_train, y_train)
resultados["Bagging"] = evaluar_modelo(BaggingRegressor(random_state=42), X_train, y_train)
resultados["Extratree"] = evaluar_modelo(BaggingRegressor(n_estimators=100,random_state=42, base_estimator=ExtraTreeRegressor()), X_train, y_train)
resultados["AdaBoost"] = evaluar_modelo(AdaBoostRegressor(random_state=42), X_train, y_train)
resultados["gradientboost_100"] = evaluar_modelo(GradientBoostingRegressor(random_state=42), X_train, y_train)
resultados["Randomforest"] = evaluar_modelo(RandomForestRegressor(random_state=42), X_train, y_train)
resultados["XGB"] = evaluar_modelo(XGBRegressor(random_state=42), X_train, y_train)

In [27]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
elastic,0.018180,0.000998,64.971523,67.953307,0.000444,0.000402,1.000000,1.000000
Lasso,0.026909,0.001025,52.935312,52.197489,0.000657,0.000413,0.814746,0.768138
Ridge,0.041206,0.000915,52.916030,52.129459,0.001006,0.000368,0.814450,0.767136
Linear,0.005648,0.000800,52.916727,52.129458,0.000138,0.000322,0.814460,0.767136
SVR,40.968656,2.484677,26.414912,29.701416,1.000000,1.000000,0.406561,0.437086
KNN,0.015299,0.064089,11.649745,8.941582,0.000373,0.025794,0.179305,0.131584
Bagging,1.319157,0.013640,0.828703,0.442532,0.032199,0.005490,0.012755,0.006512
Extratree,4.318103,0.137840,0.660459,0.317955,0.105400,0.055476,0.010165,0.004679
AdaBoost,1.566872,0.015098,11.011742,10.939182,0.038246,0.006077,0.169486,0.160981
gradientboost_100,1.585498,0.008977,1.327268,1.131785,0.038700,0.003613,0.020428,0.016655


In [28]:
rf = RandomForestRegressor()
tree = BaggingRegressor(n_estimators=100,random_state=42, base_estimator=ExtraTreeRegressor())
bagging = BaggingRegressor()

In [29]:
rf.fit(X_train,y_train)
tree.fit(X_train,y_train)
bagging.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [30]:
y_forest = rf.predict(X_test)
y_tree = tree.predict(X_test)
y_bagging = bagging.predict(X_test)

In [31]:
from sklearn import metrics

In [32]:
forest_MAE = metrics.mean_absolute_error(y_test, y_forest)
forest_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_forest))
forest_R2 = metrics.r2_score(y_test, y_forest)
tree_MAE = metrics.mean_absolute_error(y_test, y_tree)
tree_RSE = np.sqrt(metrics.mean_squared_error(y_test, y_tree))
tree_R2 = metrics.r2_score(y_test, y_tree)
bagging_MAE = metrics.mean_absolute_error(y_test, y_bagging)
bagging_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_bagging))
bagging_R2 = metrics.r2_score(y_test, y_bagging)

In [33]:
errores = {
    "Random_forest": [forest_MAE,forest_RMSE,forest_R2],
    "ExtraTree": [tree_MAE,tree_RSE,tree_R2],
    "Bagging": [bagging_MAE,bagging_RMSE,bagging_R2]
    
}

In [34]:
errores

{'Random_forest': [0.2644758230929679, 1.689227153960198, 0.9998288489732485],
 'ExtraTree': [0.1725379880857512, 1.1815559705454168, 0.9999162640409053],
 'Bagging': [0.2573464428392854, 1.2029853610647998, 0.9999131991284456]}

In [35]:
error = pd.DataFrame(data=errores)

In [36]:
error = error.rename(index={0: 'MAE', 1:"RMSE", 2: "R2"})

# Comparing the best regressors

In [37]:
error


,Random_forest,ExtraTree,Bagging
MAE,0.264476,0.172538,0.257346
RMSE,1.689227,1.181556,1.202985
R2,0.999829,0.999916,0.999913


# Importing the results of the Artifical neural network created in Artificial neural network file

In [38]:
from sklearn.externals import joblib

In [39]:
ann = joblib.load('ann2años.pkl')

In [40]:
y_ann = ann.predict(X_test) 

In [41]:
ann_MAE = metrics.mean_absolute_error(y_test, y_ann)
ann_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_ann))
ann_R2 = metrics.r2_score(y_test, y_ann)

In [42]:
error["ANN"] = ann_MAE, ann_RMSE, ann_R2

In [60]:
error

,Random_forest,ExtraTree,Bagging,ANN
MAE,0.200380,0.116444,0.200525,7.100676
RMSE,0.481428,0.251843,0.436424,10.325655
R2,0.999983,0.999995,0.999986,0.992328
